In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
import numpy as np


# Cargar el conjunto de datos
data = pd.read_csv('https://cocl.us/concrete_data')

# Dividir el conjunto de datos en características y etiquetas
X = data.drop('Strength', axis=1)  # Asumiendo que 'Strength' es la columna objetivo
y = data['Strength']

In [3]:
import os
import tensorflow as tf
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


A. Build a Baseline Model
Objective:
Create a baseline neural network model to predict the strength of concrete based on various features.

Model Architecture:

Input layer based on the number of features.
One hidden layer with 10 nodes and a ReLU activation function.
Output layer to predict concrete strength.
Adam optimizer and mean squared error (MSE) as the loss function.
Procedure:

Data is split into training and test sets, holding 30% of the data for testing.
The model is trained on the training data for 50 epochs.
The model's predictions on the test data are compared to the actual values to compute the MSE.
Steps 1-3 are repeated 50 times to create a distribution of MSEs.
The mean and standard deviation of these MSEs are reported.

In [4]:
mse_list_A = []

for _ in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

    model_A = Sequential()
    model_A.add(Dense(10, activation='relu', input_shape=(X_train.shape[1],)))
    model_A.add(Dense(1))

    model_A.compile(optimizer='adam', loss='mean_squared_error')
    model_A.fit(X_train, y_train, epochs=50, verbose=0)

    y_pred = model_A.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list_A.append(mse)

mean_mse_A = np.mean(mse_list_A)
std_mse_A = np.std(mse_list_A)

10/10 [==============================] - 0s 4ms/step


In [5]:
print(f'Parte A - Mean MSE: {mean_mse_A}')
print(f'Parte A - Standard Deviation of MSE: {std_mse_A}')


Parte A - Mean MSE: 375.37107731311875
Parte A - Standard Deviation of MSE: 453.4803552564334


A. Build a Baseline Model
Objective:
Create a baseline neural network model to predict the strength of concrete based on various features.

Model Architecture:

Input layer based on the number of features.
One hidden layer with 10 nodes and a ReLU activation function.
Output layer to predict concrete strength.
Adam optimizer and mean squared error (MSE) as the loss function.
Procedure:

Data is split into training and test sets, holding 30% of the data for testing.
The model is trained on the training data for 50 epochs.
The model's predictions on the test data are compared to the actual values to compute the MSE.
Steps 1-3 are repeated 50 times to create a distribution of MSEs.
The mean and standard deviation of these MSEs are reported.

In [6]:
# Normalizar los datos
X_norm = (X - X.mean()) / X.std()

mse_list_B = []

for _ in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=None)

    model_B = Sequential()
    model_B.add(Dense(10, activation='relu', input_shape=(X_train.shape[1],)))
    model_B.add(Dense(1))

    model_B.compile(optimizer='adam', loss='mean_squared_error')
    model_B.fit(X_train, y_train, epochs=50, verbose=0)

    y_pred = model_B.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list_B.append(mse)

mean_mse_B = np.mean(mse_list_B)
std_mse_B = np.std(mse_list_B)


10/10 [==============================] - 0s 5ms/step


In [7]:
print(f'Parte B - Mean MSE: {mean_mse_B}')
print(f'Parte B - Standard Deviation of MSE: {std_mse_B}')


Parte B - Mean MSE: 379.131004856568
Parte B - Standard Deviation of MSE: 85.14128411264174


C. Increase the Number of Epochs
Objective:
Determine the effect of training the model for a longer time on its performance.

Procedure:

Using the normalized data from Part B, the model is trained for 100 epochs instead of 50.
The model's performance is then evaluated as before.
The process is repeated 50 times, and the mean and standard deviation of the MSEs are reported.
The results are compared to those from Part B to observe the impact of longer training.

In [8]:
mse_list_C = []

for _ in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=None)

    model_C = Sequential()
    model_C.add(Dense(10, activation='relu', input_shape=(X_train.shape[1],)))
    model_C.add(Dense(1))

    model_C.compile(optimizer='adam', loss='mean_squared_error')
    model_C.fit(X_train, y_train, epochs=100, verbose=0)  # 100 epochs esta vez

    y_pred = model_C.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list_C.append(mse)

mean_mse_C = np.mean(mse_list_C)
std_mse_C = np.std(mse_list_C)


10/10 [==============================] - 0s 4ms/step


In [9]:
print(f'Parte C - Mean MSE: {mean_mse_C}')
print(f'Parte C - Standard Deviation of MSE: {std_mse_C}')


Parte C - Mean MSE: 165.55179295935213
Parte C - Standard Deviation of MSE: 14.953390414362042


D. Increase the Number of Hidden Layers
Objective:
Explore how adding complexity to the model by increasing the number of hidden layers affects its performance.

Model Architecture:

Input layer based on the number of features.
Three hidden layers, each with 10 nodes and a ReLU activation function.
Output layer to predict concrete strength.
Procedure:

Using the normalized data from Part B, the new model with additional hidden layers is trained for 100 epochs.
The model's performance is evaluated as before.
This process is repeated 50 times to assess the model's stability and performance.
The mean and standard deviation of the MSEs are reported and compared to the results from Part B.

In [10]:
mse_list_D = []

for _ in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=None)

    model_D = Sequential()
    model_D.add(Dense(10, activation='relu', input_shape=(X_train.shape[1],)))
    model_D.add(Dense(10, activation='relu'))
    model_D.add(Dense(10, activation='relu'))
    model_D.add(Dense(1))

    model_D.compile(optimizer='adam', loss='mean_squared_error')
    model_D.fit(X_train, y_train, epochs=50, verbose=0)  # 50 epochs como en Parte B

    y_pred = model_D.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list_D.append(mse)

mean_mse_D = np.mean(mse_list_D)
std_mse_D = np.std(mse_list_D)


10/10 [==============================] - 0s 5ms/step


In [11]:
print(f'Parte D - Mean MSE: {mean_mse_D}')
print(f'Parte D - Standard Deviation of MSE: {std_mse_D}')


Parte D - Mean MSE: 129.38234203997735
Parte D - Standard Deviation of MSE: 14.540814062770917
